In [1]:
# pip install numpy, tensorflow, keras, pillow


<h1>1. Import the libraries and load the dataset</h1>
First, we are going to import all the modules that we are going to need for training our model. The Keras library already contains some datasets and MNIST is one of them. So we can easily import the dataset and start working with it. The mnist.load_data() method returns us the training data, its labels and also the testing data and its labels.

In [2]:
# Importing necessary libraries for model training and GUI creation

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist

# Libraries for GUI
import tkinter as tk
from tkinter import Canvas
from PIL import Image, ImageDraw

<h1> 2. Loading and Pre-processing the Data

In [3]:
# Loading and preprocessing the MNIST dataset

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = tf.keras.utils.to_categorical(train_labels)
test_labels = tf.keras.utils.to_categorical(test_labels)

<h1>3. Create the model</h1>
Now we will create our CNN model in Python data science project. A CNN model generally consists of convolutional and pooling layers. It works better for data that are represented as grid structures, this is the reason why CNN works well for image classification problems. The dropout layer is used to deactivate some of the neurons and while training, it reduces offer fitting of the model. We will then compile the model with the Adadelta optimizer.

In [4]:
# Building the CNN model

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

# Compiling the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Training the model
model.fit(train_images, train_labels, epochs=5, batch_size=64)

# Optionally, save the model for later use
# model.save('mnist_model.h5')


Epoch 1/5
938/938 [==============================] - 8s 9ms/step - loss: 0.1940 - accuracy: 0.9404
Epoch 2/5
938/938 [==============================] - 9s 9ms/step - loss: 0.0515 - accuracy: 0.9844
Epoch 3/5
938/938 [==============================] - 10s 11ms/step - loss: 0.0358 - accuracy: 0.9889
Epoch 4/5
938/938 [==============================] - 10s 11ms/step - loss: 0.0290 - accuracy: 0.9910
Epoch 5/5
938/938 [==============================] - 10s 11ms/step - loss: 0.0236 - accuracy: 0.9924


<h1>Evaluate the model</h1>
We have 10,000 images in our dataset which will be used to evaluate how good our model works. The testing data was not involved in the training of the data therefore, it is new data for our model. The MNIST dataset is well balanced so we can get around 99% accuracy.

In [5]:
# Evaluating model accuracy on the test dataset

test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_acc*100:.2f}%")

313/313 [==============================] - 1s 4ms/step - loss: 0.0321 - accuracy: 0.9903
Test accuracy: 99.03%


<h1>Create GUI to predict digits</h1>
Need to run the 'gui_digit_recognizer.py' through command line

In [7]:
# GUI for digit recognition

import tkinter as tk
from tkinter import Canvas
from PIL import Image, ImageDraw

class DigitRecognizer(tk.Tk):
    def __init__(self, model):
        super().__init__()

        self.model = model
        self.canvas_width = 280
        self.canvas_height = 280
        self.pen_width = 15
        self.initialize_ui()

    def initialize_ui(self):
        self.canvas = Canvas(self, width=self.canvas_width, height=self.canvas_height, bg="white")
        self.canvas.pack(pady=20)

        self.canvas.bind("<B1-Motion>", self.paint)

        clear_button = tk.Button(self, text="Clear Canvas", command=self.clear_canvas)
        clear_button.pack(pady=20)

        recognize_button = tk.Button(self, text="Recognize Digit", command=self.recognize_digit)
        recognize_button.pack(pady=20)

        self.label = tk.Label(self, text="Draw a digit...", font=("Helvetica", 16))
        self.label.pack(pady=20)

    def paint(self, event):
        x, y = event.x, event.y
        self.canvas.create_oval((x, y, x + self.pen_width, y + self.pen_width), fill="black", width=0)

    def clear_canvas(self):
        self.canvas.delete("all")

    def recognize_digit(self):
        # Get the canvas content as an image
        canvas_image = Image.new("RGB", (self.canvas_width, self.canvas_height), "white")
        draw = ImageDraw.Draw(canvas_image)
        for item in self.canvas.find_all():
            x0, y0, x1, y1 = self.canvas.coords(item)
            draw.ellipse([x0, y0, x1, y1], fill="black")

        digit, confidence = self.predict_digit(canvas_image)
        self.label.config(text=f"Predicted Digit: {digit} (Confidence: {confidence:.2f}%)")

    def predict_digit(self, img):
        img = img.resize((28, 28))
        img = img.convert('L')
        img = np.array(img)
        img = img.reshape(1, 28, 28, 1)
        img = img / 255.0
        prediction = self.model.predict([img])[0]
        return np.argmax(prediction), max(prediction) * 100

# Running the GUI
app = DigitRecognizer(model)
app.title("Handwritten Digit Recognizer")
app.mainloop()

1/1 [==============================] - 0s 13ms/step


SystemExit: 0

/Users/ayushpatel/Library/Python/3.9/lib/python/site-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


: 

In [ ]:
# from keras.models import load_model
# from tkinter import *
# import tkinter as tk
# # import win32gui
# from PIL import ImageGrab, Image
# import numpy as np

# model = load_model('mnist.h5')

# def predict_digit(img):
#     #resize image to 28x28 pixels
#     img = img.resize((28,28))
#     #convert rgb to grayscale
#     img = img.convert('L')
#     img = np.array(img)
#     #reshaping to support our model input and normalizing
#     img = img.reshape(1,28,28,1)
#     img = img/255.0
#     #predicting the class
#     res = model.predict([img])[0]
#     return np.argmax(res), max(res)

# class App(tk.Tk):
#     def __init__(self):
#         tk.Tk.__init__(self)

#         self.x = self.y = 0

#         # Creating elements
#         self.canvas = tk.Canvas(self, width=300, height=300, bg = "white", cursor="cross")
#         self.label = tk.Label(self, text="Thinking..", font=("Helvetica", 48))
#         self.classify_btn = tk.Button(self, text = "Recognise", command =         self.classify_handwriting) 
#         self.button_clear = tk.Button(self, text = "Clear", command = self.clear_all)

#         # Grid structure
#         self.canvas.grid(row=0, column=0, pady=2, sticky=W, )
#         self.label.grid(row=0, column=1,pady=2, padx=2)
#         self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
#         self.button_clear.grid(row=1, column=0, pady=2)

#         #self.canvas.bind("<Motion>", self.start_pos)
#         self.canvas.bind("<B1-Motion>", self.draw_lines)

#     def clear_all(self):
#         self.canvas.delete("all")

#     def classify_handwriting(self):
#         HWND = self.canvas.winfo_id() # get the handle of the canvas
#         rect = tk.GetWindowRect(HWND) # get the coordinate of the canvas
#         im = ImageGrab.grab(rect)

#         digit, acc = predict_digit(im)
#         self.label.configure(text= str(digit)+', '+ str(int(acc*100))+'%')

#     def draw_lines(self, event):
#         self.x = event.x
#         self.y = event.y
#         r=8
#         self.canvas.create_oval(self.x-r, self.y-r, self.x + r, self.y + r, fill='black')

# app = App()
# mainloop()